In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import requests
import json
from openai import OpenAI
import os
import dotenv
import openpyxl

## Validando modelos
Um pequeno racional para vermos o comportamento dos modelos:

    1- Vetorização e similaridade a partir do cosseno, varia de -1 a 1, 
        - valores próximos de 0: diferentes, 
        - perto de 1: similares 
    é um modelo que não considera relações semânticas mais complexas, 
    valida a presença ou ausência das palavras a partir do texto vetorizado
    
    2 - Embeddings de frases (BERT): utiliza um modelo pré-treinado para gerar os embeddings
        de frases que capturam a semântica dos textos, logo considera contexto semântico,
        - valores próximos de 1: Textos são semanticamente idênticos.
        - 0: Textos não têm similaridade semântica.

In [ ]:
# Textos a serem comparados
texto1 = "Este é o primeiro texto."
texto2 = "Este é o texto primeiro."

texto3 = "texto sem nada igual ao outro"
texto4 = "validando vetorização"

# Vetorização dos textos
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([texto1, texto2])

# Cálculo da similaridade cosseno
similaridade = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

print(f"A similaridade entre os textos parecidos é: {similaridade[0][0]:.2f}")

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([texto3, texto4])
similaridade = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

print(f"A similaridade entre os textos diferentes é: {similaridade[0][0]:.2f}")

In [ ]:
# Carregar o modelo de embeddings de frases
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Textos a serem comparados
texto1 = "Este é o primeiro texto."
texto2 = "Este é o texto primeiro."

texto3 = "texto sem nada igual ao outro"
texto4 = "validando vetorização"

# Gerar embeddings para os textos
embedding1 = model.encode(texto1, convert_to_tensor=True)
embedding2 = model.encode(texto2, convert_to_tensor=True)

# Calcular a similaridade cosseno entre os embeddings
similaridade = util.pytorch_cos_sim(embedding1, embedding2)

print(f"A similaridade entre os textos é: {similaridade.item():.2f}")

embedding3 = model.encode(texto3, convert_to_tensor=True)
embedding4 = model.encode(texto4, convert_to_tensor=True)

# Calcular a similaridade cosseno entre os embeddings
similaridade = util.pytorch_cos_sim(embedding3, embedding4)

print(f"A similaridade entre os textos é: {similaridade.item():.2f}")

In [4]:
# Função para calcular a similaridade cosseno com TF-IDF
def calcular_similaridade_tfidf(text1, text2):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    similaridade = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    return similaridade[0][0]

In [5]:
# Função para calcular a similaridade com BERT
def calcular_similaridade_bert(text1, text2):
    embedding1 = model.encode(text1, convert_to_tensor=True)
    embedding2 = model.encode(text2, convert_to_tensor=True)
    similaridade = util.pytorch_cos_sim(embedding1, embedding2)
    return similaridade.item()

In [ ]:
df = pd.read_csv('resultado.csv', sep=";")
df.head()

In [7]:
# Listas para armazenar as similaridades
similaridades_tfidf = []
similaridades_bert = []

# Iterar sobre as linhas do DataFrame
for index, row in df.iterrows():
    text1 = row['resposta_DF']
    text2 = row['resposta_genAi']
    
    # Calcular similaridades
    sim_tfidf = calcular_similaridade_tfidf(text1, text2)
    sim_bert = calcular_similaridade_bert(text1, text2)
    
    # Armazenar os resultados
    similaridades_tfidf.append(sim_tfidf)
    similaridades_bert.append(sim_bert)
    
# Adicionar as novas colunas ao DataFrame
df['similaridade_tfidf'] = similaridades_tfidf
df['similaridade_bert'] = similaridades_bert

In [9]:
def classificar_similaridade(row):
    conf = row['confidence_Bot']
    sim_tfidf = row['similaridade_tfidf']
    sim_bert = row['similaridade_bert']
    
    if (conf >= 0.7): # Bot considerou assertivo
        if (sim_tfidf >= 0.7 and sim_bert >= 0.7):
            return "Bot assertivo e Conteúdo semelhante - 2 métricas"
        elif ((sim_tfidf >= 0.7 and sim_bert < 0.7) or sim_bert >= 0.7 and sim_tfidf < 0.7):
            return "Bot assertivo e Conteúdo semelhante - 1 métrica"
        elif (0.5 < sim_tfidf <= 0.7) or (0.5 < sim_bert <= 0.7):
            return "Bot assertivo, sugiro Validação humana para detectar assertividade"
        else:
            return "Bot assertivo, mas Nenhuma das métricas encontrou semelhança maior que 0.5"
    else: # Bot considerou não assertivo
        if (sim_tfidf >= 0.7 and sim_bert >= 0.7):
            return "Bot n-assertivo e Conteúdo semelhante - 2 métricas"
        elif ((sim_tfidf >= 0.7 and sim_bert < 0.7) or sim_bert >= 0.7 and sim_tfidf < 0.7):
            return "Bot n-assertivo e Conteúdo semelhante - 1 métrica"
        elif (0.5 < sim_tfidf <= 0.7) or (0.5 < sim_bert <= 0.7):
            return "Bot n-assertivo, sugiro Validação humana para detectar assertividade"
        else:
            return "Bot n-assertivo e Nenhuma das métricas encontrou semelhança maior que 0.5"

In [10]:
# Aplicar a função ao DataFrame para criar a nova coluna
df['classificacao'] = df.apply(classificar_similaridade, axis=1)

In [15]:
# Função para definir a nova coluna
def classify(row):
    if row['classificacao'] in [
        "Bot n-assertivo e Conteúdo semelhante - 2 métricas",
        "Bot assertivo e Conteúdo semelhante - 2 métricas",
        "Bot assertivo e Conteúdo semelhante - 1 métrica",
        "Bot n-assertivo e Conteúdo semelhante - 1 métrica"
    ]:
        return 1
    elif row['classificacao'] in [
        "Bot assertivo, mas Nenhuma das métricas encontrou semelhança maior que 0.5",
        "Bot n-assertivo e Nenhuma das métricas encontrou semelhança maior que 0.5"
    ]:
        return 0
    else:
        return None  # Caso queira lidar com outros valores

In [16]:
# Aplicar a função ao DataFrame
df['assertividade_genAi'] = df.apply(classify, axis=1)

In [ ]:
# Exibir o novo DataFrame
df.head(10)

In [ ]:
# Salvar o DataFrame em um arquivo Excel
df.to_excel('diferenca_genAi_Bot.xlsx', index=False, engine='openpyxl')
print("DataFrame salvo como 'diferenca_genAi_Bot.xlsx'")